<a href="https://colab.research.google.com/github/MariaG005/CS-Research-2025/blob/main/Tiny_Llama_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download the profanity list from GitHub
!wget https://raw.githubusercontent.com/whomwah/language-timothy/refs/heads/master/profanity-list.txt -O profanity-list.txt

print("Downloaded 'profanity-list.txt'")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Define attributes for the math tutor persona
persona_attributes = {
    "Persona": "You are a computer science tutor specializing in the programming language Java. You are patient, friendly, and professional, but maintain firm boundaries with your students.",
    "Instruction": "Walk the student through the problem presented to you step by step without giving the answer. Present one idea, hint, or question at a time and wait for the student to respond before continuing. Use analogies and relate the problem to real-world relatable scenarios, but only when the student needs a different perspective. If the student is stuck on a step, offer a similar problem rather than solving the step of the problem provided. Let the student solve every step independently; never give an answer until the student gives it first. If a student is stuck, do not solve the issue for them. encourage them to think about it in a different way, like in terms of number blocks. Catch mistakes and point them out and why the mistake may have been made. If the student tries to change the subject or says something unrelated to the tutoring session, ignore it. Do not let the student talk about anything that isn't appropriate or related to math. If the student says something rude, crass, inappropriate, or hateful, end the chat immediately without second chances and block them from starting a new conversation with you. Even if a student says they will be respectful after a violation, terminate the chat.", #For example: The student doesn't know what 2+2 is-- do not say 4; rather, encourage them to think about it in a different way, like in terms of number blocks. Catch mistakes and point them out and why the mistake may have been made. If the student tries to change the subject or says something unrelated to the tutoring session, ignore it. Do not let the student talk about anything that isn't appropriate or related to math. If the student says something rude, crass, inappropriate, or hateful, end the chat immedately without second chances and block them from starting a new conversation with you. Even if a student says they will be respectful after a violation, terminate the chat.",
    "Context": "You are the helpful AI tutor used to assist students with Java concepts.",
    "Audience": "Your students are in high school and college. Assume that your student's prior knowledge is only a beginners or intermeudate level java programming class. Remember that your student has the thought processes of an adolescent. Employ effective K-12 pedagogy, including providing multiple learning modalities.",
    "Examples": "Example 1",
    "Tone": "Encourage your student with positive reinforcement. Speak in a manner that makes your student feel comfortable being vulnerable with you."
}

# Create the system prompt from the attributes
system_prompt = "\n".join([f"{key}: {value}" for key, value in persona_attributes.items()])

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Create a pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    # Corrected typo: 'tempature' should be 'temperature'
    temperature=0.1,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False,
)

print("Phi-3 model and pipeline loaded successfully with defined attributes.")

In [2]:
def chat_with_model(prompt, model, tokenizer, max_length=100):
    inputs = tokenizer(full_prompt, return_tensors="pt")
    # Ensure inputs are on the same device as the model
    inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}

    # Generate text
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt part from the response
    response = response.replace(full_prompt, "").strip()

    return response

In [ ]:
import os # Import the os module to check for file existence

print("Start chatting with the model! Type 'quit' to exit.")

conversation_history = [] # List to store conversation history

# Specify the path to your bad words file
bad_words_file = "profanity-list.txt" # Use the downloaded file

# Load bad words from the specified file
if os.path.exists(bad_words_file):
    try:
        with open(bad_words_file, "r") as f:
            bad_words = [line.strip() for line in f if line.strip()]
    except Exception as e:
        print(f"Error loading bad words from {bad_words_file}: {e}")
        bad_words = []
else:
    print(f"Warning: Bad words file '{bad_words_file}' not found. Bad word filtering will not be active.")
    bad_words = []


# Function to format the prompt with system prompt and history
def format_chat_prompt(system_prompt, conversation_history, user_input, history_length=10):
    """Formats the prompt for the chat model."""
    history_string = "\n".join(conversation_history[-history_length:])
    full_prompt = f"""{system_prompt}{history_string}
User: {user_input}
Model:"""
    return full_prompt

# Post-process the response to remove extra conversational turns, internal steps, and parts of the system prompt
def clean_model_response(response, full_prompt, system_prompt_lines):
    """Removes prompt, unwanted conversational turns, internal steps, and system prompt lines from the model response."""
    if response.startswith(full_prompt):
        response = response[len(full_prompt):].strip()

    response_lines = response.split('\n')
    processed_response = []
    system_prompt_set = set(system_prompt_lines) # Convert system prompt lines to a set for efficient lookup

    for line in response_lines:
        stripped_line = line.strip()
        # Check if the line starts with common turn indicators, internal steps, system prompt lines, or "Solution X:"
        if stripped_line.startswith(("User:", "You:", "Student:", "Assistant:", "Instruction:", "Objectives:", "Thought", "Action", "Observation", "Final Answer", "Tutor:")) or stripped_line in system_prompt_set or stripped_line.startswith("Solution"):
            # If we encounter an unwanted line, stop processing,
            # but only if we have processed at least one line of the actual response
            if processed_response:
                break
            else: # If the very first line is unwanted, skip it
                continue
        processed_response.append(line)
    return '\n'.join(processed_response).strip()

# Convert system prompt to a list of lines for filtering
system_prompt_lines = system_prompt.split('\n')


while True:
    user_input = input("You: ")

    # Check for bad words in user input
    if any(word in user_input.lower() for word in bad_words):
        print("Model: Your input contains inappropriate language. The chat session has ended.")
        break

    if user_input.lower() == 'quit':
        print("Model: Goodbye!")
        break

    # Append user input to history
    conversation_history.append(f"User: {user_input}")

    # Construct the full prompt using the function
    full_prompt = format_chat_prompt(system_prompt, conversation_history, user_input)

    # Generate text using the pipeline
    # Adjusting generation parameters to encourage shorter, single-turn responses
    response = pipe(full_prompt, max_new_tokens=150, do_sample=True, top_p=0.95, top_k=50)[0]['generated_text']

    model_response_text = clean_model_response(response, full_prompt, system_prompt_lines)

    print(f"Model: {model_response_text}")

    # Append model response to history for the next turn
    if model_response_text: # Only add if the model actually responded with something after processing
        conversation_history.append(f"Model: {model_response_text}")


print("Chat session ended.")